## 파이토치 실습

### 1. 파이토치로 선형 회귀 구현하기

#### 선형회귀 가설식

$$
H(x) = Wx+b
$$

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(777)

In [13]:
# 데이터 (y = 2x)
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [14]:
# 모델 선언 및 초기화
model = nn.Linear(1, 1) # (input_dim, output_dim)

In [15]:
# 모델의 초기화된 가중치와 편향 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.8362]], requires_grad=True), Parameter containing:
tensor([-0.0177], requires_grad=True)]


--------------------------
#### 1) 옵티마이저 - 경사 하강법(Gradient Descent)

* 손실 함수(Loss function): 일반적으로 평균 제곱 오차 (Mean Squared Error, MSE) 사용
* 손실 함수(Loss function) = 비용 함수(Cost function) = 오차 함수(Error function) = 목적 함수 (Objective function)

<center>

$L(w,b)= \sum_{i=1}^{n} (y^{(i)}-\hat{y}^{(i)})^2$

</center>


In [5]:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/Hyelim-Shin/AIdev/master/Images/10-1.GD.png', width=500)

$$
gradient = \frac{\partial L(W)}{\partial W}
$$

* 기울기가 음수일 때 (Negative gradient): W의 값이 증가

$$
W := W-\alpha \times (-gradient) = W + \alpha \times gradient
$$

* 기울기가 양수일 때 (Positive gradient): W의 값이 감소

$$
W := W-\alpha \times gradient
$$


In [20]:
# 다양한 최적화 알고리즘들
# [출처] https://ganghee-lee.tistory.com/24
Image(url='https://raw.githubusercontent.com/Hyelim-Shin/AIdev/master/Images/10-1.optimizer.png', width=500)

##### 자동 미분 (Autograd)

In [16]:
w = torch.tensor(2.0, requires_grad=True)

y = w**2
z = 2 * y + 5

In [17]:
z.backward()

In [18]:
w.grad

tensor(8.)

#### 2) 데이터 로드

전체 데이터를 하나의 행렬로 선언하여 전체 데이터에 대해서 경사 하강법을 수행하여 학습할 수 있음. 만약, 데이터가 수십만개 이상이라면 전체 데이터에 대해서 경사 하강법을 수행하는 것은 매우 느릴 뿐만 아니라 많은 계산량을 필요로 함. 따라서 전체 데이터를 더 작은 단위로 나누어서 해당 단위로 학습하는 방식을 사용함.

* 에포크(Epoch): 전체 훈련 데이터가 학습에 한 번 사용된 주기
* 미니배치(minibatch): 전체 데이터를 나누는 단위
* 배치 크기: 미니 배치의 크기
    - 보통 2의 제곱수 사용
* 이터레이션(Iteration): 한 번의 에포크 내에서 이루어지는 매개변수 $(W$와 $b)$ 업데이트 횟수

In [22]:
Image(url='https://raw.githubusercontent.com/Hyelim-Shin/AIdev/master/Images/10-1.data.png', width=500)

-----------------------

In [6]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 학습률(learning rate, lr)은 0.01로 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# 전체 훈련 데이터에 대해 경사 하강법을 2000회 반복

# 총 에포크 수 설정
n_epochs = 2000
for epoch in range(n_epochs + 1):
    # 1) 가설모델 H(x)=Wx+b 계산
    prediction = model(x_train)

    # 2) loss 계산
    loss = F.mse_loss(prediction, y_train)  # <-- 파이토치에서 제공하는 평균 제곱 오차 함수

    # 3) loss로 H(x) 개선
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 손실 함수를 미분하여 gradient 계산
    loss.backward()
    # W, b 업데이트
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Loss: {:.6f}'.format(
            epoch, n_epochs, loss.item()
        ))

In [ ]:
# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) 
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y) 

In [ ]:
print(list(model.parameters()))

* forward 연산: $H(x)$ 식에 입력 $x$로부터 예측된 $y$를 얻는 것
    - ex) 학습전 ```prediction = model(x_train)```, 학습후 ```pred_y = model(new_var)```
* backward 연산: 학습 과정에서 비용 함수를 미분하여 기울기를 구하는 것
    - ex) ```cost.backward()```

### 2. 파이토치로 로지스틱 회귀 구현하기

#### 로지스틱 회귀의 가설식

$$
H(x) = sigmoid(Wx+b)
$$

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

* ```nn.Sequential()```: 여러 함수들을 연결

In [ ]:
model = nn.Sequential(
    nn.Linear(2, 1), # input_dim=2, output_dim=1
    nn.Sigmoid() # 출력은 시그모이드 함수를 거친다
)

-------------------
#### 1) 손실 함수(Loss Function)

**평균 제곱 오차(MSE)를 로지스틱 회귀의 비용함수로 사용한다면?**

In [23]:
Image(url='https://raw.githubusercontent.com/Hyelim-Shin/AIdev/master/Images/10-1.sigmoid_mse.png', width=300)

시그모이드 함수의 출력값은 0에서 1사이. 즉, 실제값이 1일 때 예측값이 0에 가까와지면 오차 증가, 실제값이 0일 때 예측값이 1에 가까워지면 오차가 커져야 함.

$$
if y=1 \rightarrow L(H(x),y)=-log(H(x))
$$

$$
if y=0 \rightarrow L(H(x),y)=-log(1-H(x))
$$

In [19]:
Image(url='https://raw.githubusercontent.com/Hyelim-Shin/AIdev/master/Images/10-1.cross_entropy.png', width=300)

하나의 식으로 통합

$$
L(H(x),y) = -[ylogH(x)+(1-y)log(1-H(x))]
$$

$$
L(W)=-\frac{1}{n} \sum_{i=1}^n [y^{(i)} logH(x^{(i)}) + (1-y^{(i)}) log(1-H(x^{(i)}))]
$$

**크로스 엔트로피 함수 (Cross Entropy Function)**

* $k$개의 다중 클래스 분류로 일반화

$$
L(W)=-\frac{1}{n} \sum_{i=1}^n \sum_{j=1}^k y_j log(p_j)
$$

----------------------------------------------

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    loss = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, loss.item(), accuracy * 100,
        ))


In [ ]:
model(x_train)